In [2]:
import random
import math
import copy
from random import uniform, randint, shuffle, sample

import time

In [3]:
def encoding_population(initial_population):
    dict = {
        'a' : 0,
        'c' : 1,
        'g' : 2,
        't' : 3
    }
    l=[]
    for i in initial_population:
        k=''
        for j in i:
            k+=str(dict[j])
        l.append(k)
    return l

In [4]:
def insert_symbol(src_string,inserted_string,pos):
    return ''.join(src_string[:pos] + inserted_string + src_string[pos:])
# Given set of strings and population size for SCS problem

# supersequence_generate(['acg', 'cat', 'gtt','tgc'])
def supersequence_generate(set_of_strings):

    '''
        Make a copy of the set_of_strings parameter for maintaining the original
        set
    '''
    copied_set_of_strings = copy.deepcopy(set_of_strings)
    supersequence = ''.join(copied_set_of_strings.pop(random.randint(0,len(set_of_strings)-1)))

    for i in range(len(copied_set_of_strings)):
        # print("i = ",i)
        counter = 0
        for j in copied_set_of_strings[i]:
            inserted_pos = random.randint(counter,len(supersequence))
            # print("j and counter and supersequence length and inserted index",j," ",counter," ",len(supersequence)," ",inserted_pos)
            if inserted_pos == len(supersequence) or j != supersequence[inserted_pos]:
                supersequence=insert_symbol(supersequence,j,inserted_pos)
            counter = inserted_pos +1
            # print(supersequence)

    return supersequence
def population_generation(pop_size,set_of_strings):
    # l=[]
    # for i in range(pop_size):
    #     l.append(supersequence_generate(set_of_strings))
    # return l
    population = []

    while len(population) < pop_size:
        population.append(supersequence_generate(set_of_strings))

    # Find the length of the string that appears the most
    most_common_length = max(len(string) for string in population)

    # Remove strings that do not match the most common length
    population = [string for string in population if len(string) == most_common_length]

    # Continue generating strings until the length of the container equals the original population size
    while len(population) < pop_size:
        generated_supersequence = supersequence_generate(set_of_strings)
        if(len(generated_supersequence)  == most_common_length):
            population.append(generated_supersequence)

    return population
def initialization(pop_size,set_of_strings):
    initial_population =  population_generation(pop_size,set_of_strings)

    encoded_population = encoding_population(initial_population)


    return encoded_population
molecules = []
origin=initialization(10,['acg', 'cat', 'gtt','tgc'])
for original_string   in origin:
    digit_list = [int(digit) for digit in original_string]
    molecules.append(digit_list)
for mol in molecules :
    print(mol)

[3, 0, 1, 2, 1, 1, 2, 2, 0, 3, 3, 3]
[2, 3, 0, 1, 1, 2, 1, 0, 3, 2, 3, 3]
[1, 2, 3, 0, 0, 1, 3, 2, 3, 2, 3, 1]
[1, 0, 2, 1, 0, 3, 3, 2, 3, 3, 2, 1]
[0, 2, 1, 3, 2, 3, 3, 2, 1, 0, 1, 3]
[2, 3, 0, 1, 3, 1, 0, 2, 3, 3, 2, 1]
[0, 2, 1, 2, 1, 3, 3, 3, 0, 2, 3, 1]
[3, 2, 3, 1, 0, 3, 0, 2, 1, 1, 3, 2]
[0, 1, 0, 3, 2, 3, 3, 2, 1, 1, 2, 3]
[0, 1, 3, 2, 1, 2, 0, 3, 3, 2, 1, 3]


In [5]:
# -*- coding: utf-8 -*-
"""
Created on Sun May 24 20:52:55 2020
@author: Kivanc
"""
#%%



a = time.time()
distanceMatrix = molecules

# distanceMatrix = molecules

n = len(distanceMatrix)

def calculateDistance(subsequence):
    index = subsequence[0]  # Düğümler arası uzaklık hesabı
    distance = 0
    for nextIndex in subsequence[1:]:
        distance += distanceMatrix[index][nextIndex]
        index = nextIndex
    return distance  # mesafe döndürüldü

def swap(sequence, i, j):
    temp = sequence[i]  # Düğümler arası yer değiştirme(indis) fonksiyonu
    sequence[i] = sequence[j]
    sequence[j] = temp

def localPhremone(subsequence, a, b):
    subsequence = subsequence[0][:]
    swap(subsequence, a, b)
    return (subsequence, calculateDistance(subsequence))  # subsequence listesi döndürür.

def globalPhremone(subsequence, a, b, c):
    subsequence = subsequence[0][:]
    swap(subsequence, a, b)
    swap(subsequence, b, c)
    return (subsequence, calculateDistance(subsequence))  # subsequence listesi döndürü

while True:
    numSubsequences = 10  # Alt dizi sayısı
    worstSubsequences = int(0.1 * numSubsequences)  # kötü değerli alt diziler (çok maliyetli)
    bestSubsequences = int(0.9 * numSubsequences)  # iyi değerlikli alt diziler (az maliyetli)
    alfa = 2  # geçiş metodu için gereken alfa değeri
    beta = 2  # geçiş metodu için gereken beta değeri
    passMax = 15  # geçiş metodu değişkenleri (increased passMax for higher range)
    passMin = 0  # geçiş metodu değişkenleri
    transitionProbability = 0
    passMethod = alfa * 1 / n * beta * (passMax - passMin)
    iterationSize = 100
    subsequences = []  # Alt dizi için array(liste)
    initPath = list(range(0, n))
    for i in range(numSubsequences):
        subsequence = sample(initPath, n)
        subsequences.append((subsequence, calculateDistance(subsequence)))

    subsequences.sort(key=lambda x: x[1])

    for iterationIndex in range(iterationSize):
        goSubsequence = subsequences[randint(0, bestSubsequences)]
        randomSubsequenceIndex = randint(0, passMethod)
        if random.random() < transitionProbability:
            morePowerfulSubsequence = localPhremone(goSubsequence, randint(0, n-1), randint(0, n-1))
            if subsequences[randomSubsequenceIndex][1] > morePowerfulSubsequence[1]:
                subsequences[randomSubsequenceIndex] = morePowerfulSubsequence
        else:
            for i in range(numSubsequences - worstSubsequences, numSubsequences):
                subsequences[i] = localPhremone(subsequences[i], randint(0, n-1), randint(0, n-1))
            subsequences.sort(key=lambda x: x[1])
        lowCostSubsequence = subsequences[0]
        effectlyGlobalSubsequence = globalPhremone(
            lowCostSubsequence, randint(0, n-1), randint(0, n-1), randint(0, n-1))
        if subsequences[0][1] > effectlyGlobalSubsequence[1]:
            subsequences[0] = effectlyGlobalSubsequence
        subsequences.sort(key=lambda x: x[1])

    subsequenceCosts = []
    for i in range(len(subsequences[0][0]) - 1):
        current_city = subsequences[0][0][i]
        next_city = subsequences[0][0][i + 1]
        cost = molecules[current_city][next_city]
        subsequenceCosts.append(cost)

    unique_elements = set(subsequenceCosts)
    if set([0, 1, 2, 3]).issubset(unique_elements):
        break


print("Subsequence", subsequenceCosts)


Subsequence [0, 0, 3, 2, 1, 0, 0, 1, 0]
